# Compute probabilities of escape
In some experiments that involve antibody selections, it is possible to spike in a "neutralization standard", which is a set of variants known not to be affected by the antibody.
In such cases, it is then possible to compute the probability of escape of each variant, which is just its change in frequency relative to the standard.
For instance, if such experiments are done at enough concentrations, it is even possible to reconstruct a conventional neutralization curve.

This notebook illustrates how to use `dms_variants` to compute these probabilities of escape:

First, import Python modules:

In [1]:
import io
import textwrap

import altair as alt

import dms_variants.codonvarianttable

import pandas as pd

Read in the `CodonVariantTable`.
These data correspond to snippets of the variant counts from a real experiment on the SARS-CoV-2 spike:

In [2]:
with open("spike.txt") as f:
    spike_seq = f.read().strip()

variants = dms_variants.codonvarianttable.CodonVariantTable.from_variant_count_df(
    variant_count_df_file="prob_escape_codon_variant_table.csv",
    primary_target="spike",
    geneseq=spike_seq,
    allowgaps=True,
)

Set up a data frame giving the antibody / no-antibody sample pairings for each selection:

In [3]:
selections_df = pd.read_csv(
    io.StringIO(
        textwrap.dedent(
            """\
    library,antibody_sample,no-antibody_sample
    lib1,thaw-1_REGN10933_0.037_1,thaw-1_no-antibody_control_1
    lib1,thaw-1_REGN10933_0.15_1,thaw-1_no-antibody_control_1
    lib1,thaw-1_REGN10933_0.15_2,thaw-1_no-antibody_control_2
    lib1,thaw-1_REGN10933_0.59_1,thaw-1_no-antibody_control_1
    lib1,thaw-1_REGN10933_0.59_2,thaw-1_no-antibody_control_2
    lib1,thaw-2_279C_0.00088_1,thaw-2_no-antibody_control_1
    lib1,thaw-2_279C_0.00088_2,thaw-2_no-antibody_control_2
    lib1,thaw-2_279C_0.0035_1,thaw-2_no-antibody_control_1
    lib1,thaw-2_279C_0.0035_2,thaw-2_no-antibody_control_2
    lib1,thaw-2_279C_0.014_1,thaw-2_no-antibody_control_1
    lib1,thaw-2_279C_0.014_2,thaw-2_no-antibody_control_2
    lib2,thaw-1_REGN10933_0.037_1,thaw-1_no-antibody_control_1
    lib2,thaw-1_REGN10933_0.037_2,thaw-1_no-antibody_control_2
    lib2,thaw-1_REGN10933_0.15_1,thaw-1_no-antibody_control_1
    lib2,thaw-1_REGN10933_0.15_2,thaw-1_no-antibody_control_2
    lib2,thaw-1_REGN10933_0.59_1,thaw-1_no-antibody_control_1
    lib2,thaw-1_REGN10933_0.59_2,thaw-1_no-antibody_control_2
    """
        )
    )
)

Now run `CodonVariantTable.prob_escape` using the above data frame to define which samples to pair in the comparisons:

In [4]:
prob_escape, neut_standard_fracs, neutralization = variants.prob_escape(
    selections_df=selections_df
)

First look at the returned `neut_standard_fracs` data frame, which gives the fraction of all reads corresponding to the neutralization standard for all samples:

In [5]:
neut_standard_fracs.round(4)

,library,antibody_sample,no-antibody_sample,antibody_count,antibody_frac,no-antibody_count,no-antibody_frac
0,lib1,thaw-1_REGN10933_0.037_1,thaw-1_no-antibody_control_1,11107,0.0210,16475,0.0098
1,lib1,thaw-1_REGN10933_0.15_1,thaw-1_no-antibody_control_1,18045,0.0220,16475,0.0098
2,lib1,thaw-1_REGN10933_0.15_2,thaw-1_no-antibody_control_2,14283,0.0371,11343,0.0103
3,lib1,thaw-1_REGN10933_0.59_1,thaw-1_no-antibody_control_1,26027,0.0636,16475,0.0098
4,lib1,thaw-1_REGN10933_0.59_2,thaw-1_no-antibody_control_2,23240,0.0510,11343,0.0103
5,lib1,thaw-2_279C_0.00088_1,thaw-2_no-antibody_control_1,8231,0.0157,11134,0.0157
6,lib1,thaw-2_279C_0.00088_2,thaw-2_no-antibody_control_2,8763,0.0177,9002,0.0147
7,lib1,thaw-2_279C_0.0035_1,thaw-2_no-antibody_control_1,9298,0.0200,11134,0.0157
8,lib1,thaw-2_279C_0.0035_2,thaw-2_no-antibody_control_2,9394,0.0231,9002,0.0147
9,lib1,thaw-2_279C_0.014_1,thaw-2_no-antibody_control_1,67940,0.1408,11134,0.0157


Now plot the fraction of all reads corresponding to the neutralization standard for all samples, making an interactive `altair` plot where you can mouse over points for details and select which libraries to show with the drop down at the bottom:

In [6]:
# make tidy version of neut_standard_fracs
melt_cols = ["antibody_frac", "no-antibody_frac"]
neut_standard_fracs_tidy = (
    neut_standard_fracs
    .melt(
        id_vars=[c for c in neut_standard_fracs.columns if c not in melt_cols],
        value_vars=melt_cols,
        value_name="neut_standard_frac",
        var_name="sample_type",
    )
    .assign(
        sample_type=lambda x: x["sample_type"].str.replace("_frac", ""),
        library_sample=lambda x: x["library"] + " " + x["antibody_sample"],
    )
)
neut_standard_fracs_tidy.head()

,library,antibody_sample,no-antibody_sample,antibody_count,no-antibody_count,sample_type,neut_standard_frac,library_sample
0,lib1,thaw-1_REGN10933_0.037_1,thaw-1_no-antibody_control_1,11107,16475,antibody,0.021003,lib1 thaw-1_REGN10933_0.037_1
1,lib1,thaw-1_REGN10933_0.15_1,thaw-1_no-antibody_control_1,18045,16475,antibody,0.021955,lib1 thaw-1_REGN10933_0.15_1
2,lib1,thaw-1_REGN10933_0.15_2,thaw-1_no-antibody_control_2,14283,11343,antibody,0.037117,lib1 thaw-1_REGN10933_0.15_2
3,lib1,thaw-1_REGN10933_0.59_1,thaw-1_no-antibody_control_1,26027,16475,antibody,0.063556,lib1 thaw-1_REGN10933_0.59_1
4,lib1,thaw-1_REGN10933_0.59_2,thaw-1_no-antibody_control_2,23240,11343,antibody,0.051002,lib1 thaw-1_REGN10933_0.59_2


In [7]:
# set up selections over other columns of interest, in this case just library:
selections = [
    alt.selection_single(
        fields=[col],
        bind=alt.binding_select(
            options=[None] + neut_standard_fracs_tidy[col].unique().tolist(),
            labels=["all"] + [str(x) for x in neut_standard_fracs_tidy[col].unique()],
            name=col,
        )
    )
    for col in ["library"]
]

neut_standard_fracs_chart = (
    alt.Chart(neut_standard_fracs_tidy)
    .encode(
        x=alt.X(
            "neut_standard_frac",
            title="neutralization standard fraction",
        ),
        y=alt.Y("library_sample", title=None),
        color="sample_type",
        shape="sample_type",
        tooltip=[
            alt.Tooltip(c, format=".2g") if c == "neut_standard_frac" else c
            for c in neut_standard_fracs_tidy.columns
        ]
    )
    .mark_point()
    .properties(width=275, height=alt.Step(14))
    .add_selection(*selections)
)
for selection in selections:
    neut_standard_fracs_chart = neut_standard_fracs_chart.transform_filter(selection)

neut_standard_fracs_chart

alt.Chart(...)